In [1]:
from datetime import datetime, timedelta, date
from calendar import monthrange
# from pytz import timezone
import pandas as pd
import numpy as np
#import feather
import datetime 
import pytz
import csv
import os
import seaborn as sns
import matplotlib.pyplot as plt
import logging

import xgboost as xgb
import time
from sklearn.pipeline import *

from math import sqrt
from numpy import loadtxt
from itertools import product
from tqdm import tqdm
import datetime
from sklearn import preprocessing
from xgboost import plot_tree

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.feature_extraction.text import TfidfVectorizer

kernel_with_output = False


from utils import *

%matplotlib inline

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [2]:
# https://www.kaggle.com/szhou42/predict-future-sales-top-11-solution

# scp jini:/home/ubuntu/future-store-sales/submissions/xgb_base_submission.csv ~/Documents/reading_training/future-store-sales/submissions/.

In [2]:
DATA_DIR = './data/'
RESOURCES_DIR = './res/'
SUBMISSION_DIR = './submissions'
MODEL_DIR = './models'

sales_train    = pd.read_csv(os.path.join(DATA_DIR, 'sales_train.csv.gz'))
items           = pd.read_csv(os.path.join(DATA_DIR, 'items.csv'))
item_categories = pd.read_csv(os.path.join(DATA_DIR, 'item_categories.csv'))
shops           = pd.read_csv(os.path.join(DATA_DIR, 'shops.csv'))
test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv.gz')) 
sample_submission = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv.gz'))


date_df = pd.read_pickle(os.path.join(RESOURCES_DIR, "russian_holidays.pkl"))
geography = pd.read_csv(os.path.join(RESOURCES_DIR, 'geography.csv'))
#holidays = feather.read_dataframe(os.path.join(ADD_RESOURCES, 'hol.feather'))

In [44]:
# For every month we create a grid from all shops/items combinations from that month

grid = []
for block_num in sales_train.date_block_num.unique():
    cur_shops = sales_train[sales_train.date_block_num == block_num]['shop_id'].unique()
    cur_items = sales_train[sales_train.date_block_num == block_num]['item_id'].unique()
    
    # cartesian product of shops, items with the block number 
    l = list( product(*[cur_shops, cur_items, [block_num]]) ) # sample l[0:2] -> [(45, 13315, 33), (45, 13881, 33)]
    
    grid.append(np.array(l , dtype='int32'))
    # np-array looks like this:
        #     array([[   45, 13315,    33],
        #        [   45, 13880,    33],
        #        [   45, 13881,    33],
        #        ...,
        #        [   21,  7640,    33],
        #        [   21,  7632,    33],
        #        [   21,  7440,    33]], dtype=int32)
    # right now, 'grid' -> is a list of np-arrays

index_cols = ['shop_id', 'item_id', 'date_block_num']
# np.vstack is used to unpack the rows from all np.arrays and put into a single array
grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32) 

In [45]:
grid.head()

,shop_id,item_id,date_block_num
0,59,22154,0
1,59,2552,0
2,59,2554,0
3,59,2555,0
4,59,2564,0


In [46]:
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [47]:
# Aggregations
sales_train['item_cnt_day'] = sales_train['item_cnt_day'].clip(0,20) # we say no-item sells more than 20 everyday

groups = sales_train.groupby(['shop_id', 'item_id', 'date_block_num'])

trainset = groups.agg({'item_cnt_day':'sum', 'item_price':'mean'}).reset_index()
trainset = trainset.rename(columns = {'item_cnt_day' : 'item_cnt_month',
                                      'item_price' : 'avg_item_price'})

trainset['item_cnt_month'] = trainset['item_cnt_month'].clip(0,20)

trainset = pd.merge(grid,trainset,how='left',on=index_cols)
trainset.item_cnt_month = trainset.item_cnt_month.fillna(0)

# Get category id
trainset = pd.merge(trainset, items[['item_id', 'item_category_id']], on = 'item_id')
trainset.to_csv('trainset_with_grid.csv', index_label=False)

trainset.head()

,shop_id,item_id,date_block_num,item_cnt_month,avg_item_price,item_category_id
0,59,22154,0,1.0,999.0,37
1,25,22154,0,5.0,999.0,37
2,24,22154,0,1.0,999.0,37
3,23,22154,0,0.0,NaN,37
4,19,22154,0,0.0,NaN,37


In [49]:
#trainset = pd.read_csv("./trainset_with_grid.csv")
trainset.head()

,shop_id,item_id,date_block_num,item_cnt_month,avg_item_price,item_category_id
0,59,22154,0,1.0,999.0,37
1,25,22154,0,5.0,999.0,37
2,24,22154,0,1.0,999.0,37
3,23,22154,0,0.0,NaN,37
4,19,22154,0,0.0,NaN,37


In [50]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


#### Benchmark submission

In [51]:
submission_filename = "benchmark_submission.csv"

last_month = trainset[trainset.date_block_num==33]
last_month = last_month.groupby(['shop_id', 'item_id']).agg({'item_cnt_month':'mean'}).reset_index()

submission = test.merge(last_month,on=['shop_id', 'item_id'] , how='left', indicator=True)
submission['item_cnt_month'] = submission.item_cnt_month.fillna(0)
print(submission.shape)

submission.drop(columns=['shop_id', 'item_id', '_merge'], inplace=True)
submission = submission.set_index('ID')

print(submission.head())
submission.to_csv(os.path.join(SUBMISSION_DIR, submission_filename))

(214200, 5)
    item_cnt_month
ID                
0              0.0
1              0.0
2              1.0
3              0.0
4              0.0


#### Using baseline features with xgboost

In [52]:
baseline_features = ['shop_id', 'item_id', 'item_category_id', 'date_block_num']
target = 'item_cnt_month'

In [53]:
# prepare test
merged_test = test.merge(items[['item_id' , 'item_category_id']], on='item_id', how='left')
merged_test['date_block_num'] = 34

merged_test.head()

,ID,shop_id,item_id,item_category_id,date_block_num
0,0,5,5037,19,34
1,1,5,5320,55,34
2,2,5,5233,19,34
3,3,5,5232,23,34
4,4,5,5268,20,34


In [15]:
base_pipeline = make_pipeline(
    ColumnSelector(baseline_features),
    xgb.XGBRegressor(max_depth = 10, min_child_weight=0.5, subsample = 1,
                     eta = 0.3, num_round = 50, seed = 1, eval_metric='rmse')
)

In [16]:
base_pipeline.fit(trainset, trainset.item_cnt_month)

[22:50:18] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'


Pipeline(memory=None,
     steps=[('columnselector', ColumnSelector(columns=['shop_id', 'item_id', 'item_category_id', 'date_block_num'])), ('xgbregressor', XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eta=0.3, eval_metric='rmse', gamma=0,
       learning_rate=0.1, max_delta_s...=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1, silent=True,
       subsample=1))])

In [18]:
from joblib import dump, load

dump(base_pipeline, os.path.join(MODEL_DIR, "base_xgb.model"))

['./models\\base_xgb.model']

In [19]:
submission_file = "xgb_base_submission.csv"

submission = test
submission['item_cnt_month'] = base_pipeline.predict(merged_test)
submission = submission.set_index('ID')
submission.drop(columns=['item_id', 'shop_id'], inplace=True)
submission.to_csv(os.path.join(SUBMISSION_DIR, submission_filename))

##### Fix category

In [54]:
# from py_translator import Translator

# l_cat = list(item_categories.item_category_name)
# l_cat_en = []
# l_cat_tr = Translator().translate(l_cat, dest='en')
# for tr in l_cat_tr:
#     l_cat_en.append(tr.text)
    

l_cat = list(item_categories.item_category_name)

for ind in range(0,1):
    l_cat[ind] = 'PC Headsets / Headphones'
for ind in range(1,8):
    l_cat[ind] = 'Access'
l_cat[8] = 'Tickets (figure)'
l_cat[9] = 'Delivery of goods'
for ind in range(10,18):
    l_cat[ind] = 'Consoles'
for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'
l_cat[25] = 'Accessories for games'
for ind in range(26,28):
    l_cat[ind] = 'phone games'
for ind in range(28,32):
    l_cat[ind] = 'CD games'
for ind in range(32,37):
    l_cat[ind] = 'Card'
for ind in range(37,43):
    l_cat[ind] = 'Movie'
for ind in range(43,55):
    l_cat[ind] = 'Books'
for ind in range(55,61):
    l_cat[ind] = 'Music'
for ind in range(61,73):
    l_cat[ind] = 'Gifts'
for ind in range(73,79):
    l_cat[ind] = 'Soft'
for ind in range(79,81):
    l_cat[ind] = 'Office'
for ind in range(81,83):
    l_cat[ind] = 'Clean'
l_cat[83] = 'Elements of a food'

item_categories['new_catg'] = l_cat
item_categories.head()

,item_category_name,item_category_id,new_catg
0,PC - Гарнитуры/Наушники,0,PC Headsets / Headphones
1,Аксессуары - PS2,1,Access
2,Аксессуары - PS3,2,Access
3,Аксессуары - PS4,3,Access
4,Аксессуары - PSP,4,Access


###### Build a train_test_set for combined feature engineering

In [57]:
train_set = trainset[baseline_features + [target]]

test_set = test.merge(items[['item_id' , 'item_category_id']], on='item_id', how='left')
test_set['date_block_num'] = 34
test_set ['item_cnt_month'] = -1
test_set.set_index('ID', inplace=True)

train_test_set = pd.concat([train_set, test_set], axis=0, ignore_index=True)

train_test_set = train_test_set.merge(item_categories, how='left', on='item_category_id', indicator=True)
train_test_set.drop(columns=['item_category_name', '_merge','item_category_id' ], inplace=True)

print(train_test_set.shape)
train_test_set.head()

(11128050, 5)


,shop_id,item_id,date_block_num,item_cnt_month,new_catg
0,59,22154,0,1.0,Movie
1,25,22154,0,5.0,Movie
2,24,22154,0,1.0,Movie
3,23,22154,0,0.0,Movie
4,19,22154,0,0.0,Movie


###### Idea 0: Add previous shop/item sales as feature (Lag feature)

In [58]:
def lagged_shop_item_sales(train_test_set, lag):    
    feature_name = "shop_item_sales_lag" + str(lag)
    train_test_set_c = train_test_set[['shop_id', 'item_id', 'date_block_num', 'item_cnt_month']].copy()
    train_test_set_c.date_block_num = train_test_set_c.date_block_num + lag
    train_test_set_c.rename(columns={'item_cnt_month': feature_name}, inplace=True)
    return train_test_set.merge(train_test_set_c, on=['item_id', 'shop_id', 'date_block_num'], how='left').fillna(0)

train_test_set = lagged_shop_item_sales(train_test_set, 2)
print(train_test_set.shape)
train_test_set.head()

(11128050, 6) 2019-06-30 00:04:42.924631


,shop_id,item_id,date_block_num,item_cnt_month,new_catg,shop_item_sales_lag2
0,59,22154,0,1.0,Movie,0.0
1,25,22154,0,5.0,Movie,0.0
2,24,22154,0,1.0,Movie,0.0
3,23,22154,0,0.0,Movie,0.0
4,19,22154,0,0.0,Movie,0.0


###### Idea 1: Add previous item sales as feature (Lag feature)

In [59]:
def lagged_item_sales(train_test_set, lag):    
    feature_name = "item_sales_lag" + str(lag)
    
    train_test_set_c = train_test_set.groupby(['date_block_num', 'item_id']).agg({'item_cnt_month':'sum'}).reset_index()
    train_test_set_c.rename(columns={'item_cnt_month': feature_name}, inplace=True)
    train_test_set_c.date_block_num = train_test_set_c.date_block_num + lag
    return train_test_set.merge(train_test_set_c, on=['item_id', 'date_block_num'], how='left').fillna(0)

train_test_set = lagged_item_sales(train_test_set, 2)
print(train_test_set.shape)
train_test_set.head()


(11128050, 7)


,shop_id,item_id,date_block_num,item_cnt_month,new_catg,shop_item_sales_lag2,item_sales_lag2
0,59,22154,0,1.0,Movie,0.0,0.0
1,25,22154,0,5.0,Movie,0.0,0.0
2,24,22154,0,1.0,Movie,0.0,0.0
3,23,22154,0,0.0,Movie,0.0,0.0
4,19,22154,0,0.0,Movie,0.0,0.0


###### Idea 2: Add previous shop/item price as feature (Lag feature)

In [60]:
def lagged_shop_item_price(train_test_set, lag):
    feature_name = "shop_item_price_lag" + str(lag)
    train_test_set_c = trainset.groupby(['date_block_num', 'item_id', 'shop_id']).agg({'avg_item_price':'mean'}).reset_index()
    train_test_set_c.rename(columns={'avg_item_price': feature_name}, inplace=True)
    train_test_set_c.date_block_num = train_test_set_c.date_block_num + lag
    return train_test_set.merge(train_test_set_c, on=['shop_id', 'item_id', 'date_block_num'], how='left').fillna(0)
    
train_test_set = lagged_shop_item_price(train_test_set,2)
print(train_test_set.shape)
train_test_set.head()

(11128050, 8)


,shop_id,item_id,date_block_num,item_cnt_month,new_catg,shop_item_sales_lag2,item_sales_lag2,shop_item_price_lag2
0,59,22154,0,1.0,Movie,0.0,0.0,0.0
1,25,22154,0,5.0,Movie,0.0,0.0,0.0
2,24,22154,0,1.0,Movie,0.0,0.0,0.0
3,23,22154,0,0.0,Movie,0.0,0.0,0.0
4,19,22154,0,0.0,Movie,0.0,0.0,0.0


###### Idea 3: Add previous item price as feature (Lag feature)

In [61]:
def lagged_item_price(train_test_set, lag):
    feature_name = "item_price_lag" + str(lag)
    train_test_set_c = trainset.groupby(['date_block_num', 'item_id']).agg({'avg_item_price':'mean'}).reset_index()
    train_test_set_c.rename(columns={'avg_item_price': feature_name}, inplace=True)
    train_test_set_c.date_block_num = train_test_set_c.date_block_num + lag
    return train_test_set.merge(train_test_set_c, on=['item_id', 'date_block_num'], how='left').fillna(0)
    
train_test_set = lagged_item_price(train_test_set,2)
print(train_test_set.shape)
train_test_set.head()

(11128050, 9)


,shop_id,item_id,date_block_num,item_cnt_month,new_catg,shop_item_sales_lag2,item_sales_lag2,shop_item_price_lag2,item_price_lag2
0,59,22154,0,1.0,Movie,0.0,0.0,0.0,0.0
1,25,22154,0,5.0,Movie,0.0,0.0,0.0,0.0
2,24,22154,0,1.0,Movie,0.0,0.0,0.0,0.0
3,23,22154,0,0.0,Movie,0.0,0.0,0.0,0.0
4,19,22154,0,0.0,Movie,0.0,0.0,0.0,0.0


###### Idea 6: Number of month from last sale of shop/item

In [ ]:
def months_from_lastsale_shop_item(train_test_set):
    train_test_set.loc[:,'lastsale_date_block_num'] = train_test_set.groupby(['shop_id', 'item_id'])['date_block_num'].apply(lambda x: x.sort_values()).shift(1)
    return train_test_set
 
train_test_set = months_from_lastsale_shop_item(train_test_set)


In [22]:
train_test_set.head()

,shop_id,item_id,date_block_num,item_cnt_month,new_catg,shop_item_sales_lag2,item_sales_lag2,shop_item_price_lag2,item_price_lag2
0,59,22154,0,1.0,Movie,0.0,0.0,0.0,0.0
1,25,22154,0,5.0,Movie,0.0,0.0,0.0,0.0
2,24,22154,0,1.0,Movie,0.0,0.0,0.0,0.0
3,23,22154,0,0.0,Movie,0.0,0.0,0.0,0.0
4,19,22154,0,0.0,Movie,0.0,0.0,0.0,0.0


In [41]:
train_test_set[(train_test_set.item_id == 22154) & (train_test_set.shop_id == 59)]

###### Idea 7: Number of month from last sale of item(Use info from past)

###### Idea 8: Item name (Tfidf text feature)

###### Cross Validation (Hyperparameter tuning)

In [42]:
train_test_set.loc[[1]]

,shop_id,item_id,date_block_num,item_cnt_month,new_catg,shop_item_sales_lag2,item_sales_lag2,shop_item_price_lag2,item_price_lag2
1,25,22154,0,5.0,Movie,0.0,0.0,0.0,0.0
